In [ ]:
import requests
import json
import pandas as pd
from ipywidgets import Text, Button, VBox, Output, Dropdown, HTML
from IPython.display import display

# Constants
CSV_FILE = "chat_history.csv"
API_URL = "https://71d0-84-216-200-13.ngrok-free.app/v1/chat/completions"

# Function to get chat completion from the API
def get_chat_completion(url, user_prompt):
    payload = {
        "user_prompt": user_prompt,
    }
    
    json_payload = json.dumps(payload)
    headers = {"Content-Type": "application/json"}

    try:
        response = requests.post(url, data=json_payload, headers=headers)
        response.raise_for_status()
        response_data = response.json()
        return response_data.get("choices", [{}])[0].get("message", {}).get("content", "")
    except requests.exceptions.RequestException as e:
        return f"An error occurred: {e}"

# Load chat history
try:
    chat_history_df = pd.read_csv(CSV_FILE)
except FileNotFoundError:
    chat_history_df = pd.DataFrame(columns=["ChatID", "Role", "Content"])

# Initialize chat session state
chat_names = chat_history_df["ChatID"].unique().tolist() if not chat_history_df.empty else ["Session 1"]
selected_chat = chat_names[0] if chat_names else "Session 1"

# Widgets
chat_select = Dropdown(options=chat_names, description='Chat Sessions:')
new_chat_button = Button(description="New Chat Session")
message_input = Text(description="Message:")
submit_button = Button(description="Submit", button_style="primary")
output_area = Output()

# Create new chat session
def new_chat_session(_):
    global chat_names, selected_chat
    new_name = f"Session {len(chat_names) + 1}"
    chat_names.append(new_name)
    selected_chat = new_name
    chat_select.options = chat_names
    chat_select.value = selected_chat
    display_chat_messages()

# Display chat messages
def display_chat_messages():
    output_area.clear_output()
    with output_area:
        messages = chat_history_df[chat_history_df.ChatID == selected_chat].to_dict("records")
        for message in messages:
            role = message["Role"].capitalize()
            content = message["Content"]
            display(HTML(f"<b>{role}:</b> {content}<br>"))

# Handle new message input
def handle_message_input(change=None):
    global chat_history_df, selected_chat
    user_message = message_input.value
    if not user_message:
        return
    
    # Add user message to history
    new_message = {"ChatID": selected_chat, "Role": "user", "Content": user_message}
    chat_history_df = chat_history_df.append(new_message, ignore_index=True)
    
    # Get assistant's response
    assistant_response = get_chat_completion(API_URL, user_message)
    
    # Debugging output
    print(f"Assistant's Response: {assistant_response}")
    
    if assistant_response:
        new_message = {"ChatID": selected_chat, "Role": "assistant", "Content": assistant_response}
        chat_history_df = chat_history_df.append(new_message, ignore_index=True)
    
    # Save updated chat history
    #chat_history_df.to_csv(CSV_FILE, index=False)
    
    # Refresh chat display
    display_chat_messages()
    
    # Clear the input box after submission
    message_input.value = ""

# Event listeners
new_chat_button.on_click(new_chat_session)
submit_button.on_click(handle_message_input)

# Main display
display(VBox([HTML("<h1>GPT4All - Chat Interface</h1>"), chat_select, new_chat_button, output_area, message_input, submit_button]))

# Initially display the messages
display_chat_messages()
